# Equations for proximity detection's parameters optimiszation


In [3]:
# Import libraries
import numpy as np
import itertools
import pandas as pd
import matplotlib.pyplot as plt

import ipywidgets as widgets
from IPython.display import display
from mpl_toolkits.mplot3d import *
import seaborn as sns

## Functions definition

In [11]:
# Propability functions within time t
def probability_function(Ta, Ws, Ts, t):
    return 1 - (1 - ((Ws / Ts))) ** (t /Ta)

# Probability for one scan interval
def probability_function_1scan(Ta, Ws, Ts):
    return 1 - (1 - ((Ws / Ts))) ** (Ts /Ta)

In [ ]:
# Calculate mean detect time for test file
def mean_detect_time(file_path):
    df = pd.read_csv(file_path)
    detect_times = df['time'].values
    return np.mean(detect_times)

In [ ]:
# Parameters in mS
adv_intervals = [20.0,30.0,40.0,50.0,60.0,70.0,80.0,90.0,100.0,200.0,300.0,400.0,500.0,600.0,700.0,800.0,900.0,1000.0]
scan_intervals = [700.0,1500.0,2500.0,3800.0,5500.0,7200.0,9400.0,11400.0,14300.0,52200.0,112500.0,184600.0,272400.0,367300.0,477300.0,592600.0,720000.0,857100.0]
scan_windoWs = [40.0,60.0,80.0,100.0,120.0,140.0,160.0,180.0,200.0,400.0,600.0,800.0,1000.0,1200.0,1400.0,1600.0,1800.0,2000.0]

# Calculate probabilities for the set of parameters
def calculate_probabilities(t):
    results = []
    for Ta, Ws, Ts in itertools.product(adv_intervals, scan_windoWs, scan_intervals):
        if Ws <= Ts:  
            p = probability_function(Ta, Ws, Ts, t)
            results.append((Ta, Ws, Ts, p))

    results = pd.DataFrame(results, columns=["adv_interval", "scan_window", "scan_interval", "probability"])
    #results.to_csv("ble_scan_adv_probability_" + str(t) + "ms.csv", index=False)

    return results

In [ ]:
# Find probability for given parameters, or closest match
def find_probability(Ta, Ws, Ts, results):
    # Try exact match first
    row = results[
        (results["adv_interval"] == Ta) &
        (results["scan_window"] == Ws) &
        (results["scan_interval"] == Ts)
    ]
    if not row.empty:
        return row["probability"].values[0]

    # Find closest combination
    diff = np.sqrt(
        (results["adv_interval"] - Ta)**2 +
        (results["scan_window"] - Ws)**2 +
        (results["scan_interval"] - Ts)**2
    )

    # Get index of closest parameter set
    idx_min = diff.idxmin()
    closest_row = results.loc[idx_min]

    print("No exact match found — returning closest combination:")
    print(closest_row[["adv_interval", "scan_window", "scan_interval"]].to_dict())

    return closest_row["probability"]


In [ ]:
# Calculate detect time for chosen parameters
def calculate_detect_time(Ta, Ws, Ts, target_probability):
    if Ws > Ts:
        raise ValueError("Scan window must be less than or equal to scan interval.")
    
    return (np.log(1 - target_probability) / np.log(1 - (Ws / Ts)) * Ta)/1000

In [ ]:
# Calculate real probability at time t from test data
def calculate_real_probability_at_t(file_path, t):
    df = pd.read_csv(file_path)
    
    total_successful_tests = len(df)
    
    if total_successful_tests == 0:
        return 0

    detections_by_time_t = df[df['time'] <= t + 0.1 * t].shape[0]
    
    probability = detections_by_time_t / total_successful_tests
    
    return probability


In [6]:
# Advertising current consumption (values based on online tools)
adv_current = {
    20.0: 168.0,
    30.0: 121.0,
    40.0: 95.0,
    50.0: 78.0,
    60.0: 66.0,
    70.0: 58.0,
    80.0: 51.0,
    90.0: 46.0,
    100.0: 42.0,
    200.0: 23.0,
    300.0: 16.0,
    400.0: 13.0,
    500.0: 11.0,
    600.0: 9.8,
    700.0: 8.8,
    800.0: 8.1,
    900.0: 7.5,
    1000.0: 7.0
}

scan_current = 3.0  # mA during scanning

# Current estimation function in uA
def current_estimation(adv_interval, scan_window, scan_interval):
    current = adv_current[adv_interval] + scan_current * (scan_window / scan_interval) * 1000.0
    return np.floor(current * 100) / 100.0


## Interactive probability calculator

In [7]:
# Interactive parameter choice
adv_interval_widget = widgets.SelectionSlider(options=adv_intervals, value=adv_intervals[0], description="Adv Interval (ms)")
scan_window_widget = widgets.SelectionSlider(options=scan_windoWs, value=scan_windoWs[0], description="Scan Window (ms)")
scan_interval_widget = widgets.SelectionSlider(options=scan_intervals, value=scan_intervals[0], description="Scan Interval (ms)")
observation_time_widget = widgets.FloatSlider(min=1, max=600, step=1, value=10, description="Observation Time (s)")

output = widgets.Output()
def update_probability(change):
    with output:
        output.clear_output()
        Ta = adv_interval_widget.value
        Ws = scan_window_widget.value
        Ts = scan_interval_widget.value
        observation_time = observation_time_widget.value
        results = calculate_probabilities(observation_time * 1000)
        p = find_probability(Ta, Ws, Ts, results)
        if p is not None:
            print(f"Probability of detection in {observation_time} seconds: {p*100:.2f}%")
        else:
            print("Invalid parameter combination.")

adv_interval_widget.observe(update_probability, names='value')
scan_window_widget.observe(update_probability, names='value')
scan_interval_widget.observe(update_probability, names='value')
observation_time_widget.observe(update_probability, names='value')

display(adv_interval_widget, scan_window_widget, scan_interval_widget, observation_time_widget, output)

SelectionSlider(description='Adv Interval (ms)', options=(20.0, 30.0, 40.0, 50.0, 60.0, 70.0, 80.0, 90.0, 100.…

SelectionSlider(description='Scan Window (ms)', options=(40.0, 60.0, 80.0, 100.0, 120.0, 140.0, 160.0, 180.0, …

SelectionSlider(description='Scan Interval (ms)', options=(700.0, 1500.0, 2500.0, 3800.0, 5500.0, 7200.0, 9400…

FloatSlider(value=10.0, description='Observation Time (s)', max=600.0, min=1.0, step=1.0)

Output()

## Detect and 1 scan probability calculation

In [ ]:
adv_inter = 500
scan_win = 1200
scan_int = 100000

print("Detect time", calculate_detect_time(adv_inter, scan_win, scan_int, 0.99), "s")
print("Mean detect time", mean_detect_time('adv-test_1000ms_2300-1700000ms.csv') / 1000, "s")

print("1 scan probability", probability_function_1scan(adv_inter, scan_win, scan_int))
print("Real prob from test", calculate_real_probability_at_t('adv-test_1000ms_2300-1700000ms.csv', scan_int))

Detect time 190.728482029835 s


NameError: name 'mean_detect_time' is not defined

## Search for the best parameters combinations

In [ ]:
# Set of parameters to test
adv_intervals_test = np.linspace(100.0, 1000.0, 10)
scan_window_test = np.linspace(1000.0, 5000.0, 21)
scan_interval_test = np.linspace(10000.0, 300000.0, 30)

p = 0.9

selected_params = []

# Search for parameter combinations that give probability close to p
for Ta in adv_intervals_test:
    for Ws in scan_window_test:
        for Ts in scan_interval_test:
            if Ws <= Ts:
                if(probability_function_1scan(Ta, Ws, Ts) >= p - 0.01 and probability_function_1scan(Ta, Ws, Ts) <= p + 0.01):
                    current_estimate = current_estimation(Ta, Ws, Ts)
                    selected_params.append((Ta, Ws, Ts, probability_function_1scan(Ta, Ws, Ts), current_estimate))

selected_params = pd.DataFrame(selected_params, columns=["adv_interval", "scan_window", "scan_interval", "1scan_probability", "current_uA"])
selected_params.sort_values(by=["scan_interval"],inplace=True)
print(selected_params)

# Find optimal parameters (min current) for each scan interval
idx_min_current = selected_params.groupby('scan_interval')['current_uA'].idxmin()
optimal_params_per_interval = selected_params.loc[idx_min_current]
print(optimal_params_per_interval)



     adv_interval  scan_window  scan_interval  1scan_probability  current_uA
127        1000.0       2000.0        10000.0           0.892626       607.0
98          900.0       2000.0        20000.0           0.903802       307.5
128        1000.0       2200.0        20000.0           0.902770       337.0
69          800.0       1800.0        20000.0           0.905369       278.1
40          700.0       1600.0        20000.0           0.907665       248.8
..            ...          ...            ...                ...         ...
68          700.0       1600.0       300000.0           0.898919        24.8
39          600.0       1400.0       300000.0           0.903556        23.8
11          500.0       1200.0       300000.0           0.909718        23.0
97          800.0       1800.0       300000.0           0.895313        26.1
156        1000.0       2200.0       300000.0           0.890091        29.0

[157 rows x 5 columns]
     adv_interval  scan_window  scan_interval  1scan